In [1]:
import pandas as pd

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [3]:
df = pd.read_csv(r'parsed_routes_data\all_routes_data.csv') #entire routes dataset
#df = pd.read_csv(r'parsed_routes_data\first_bulk_of_routes_data.csv') #small sample dataset for testing
df.rename(columns={'Unnamed: 0': 'route_id', 'Unnamed: 1': 'num_of_waypoint'}, inplace=True)
df

,route_id,num_of_waypoint,lat_lgt,latitude,longitude
0,1000303.gpx,0,"('38.29842', '-2.65105')",38.29842,-2.65105
1,1000303.gpx,1,"('38.29647', '-2.65079')",38.29647,-2.65079
2,1000303.gpx,2,"('38.29418', '-2.65011')",38.29418,-2.65011
3,1000303.gpx,3,"('38.29499', '-2.65174')",38.29499,-2.65174
4,1000303.gpx,4,"('38.29674', '-2.65174')",38.29674,-2.65174
...,...,...,...,...,...
14486915,99997.gpx,60,"('48.27417', '16.33581')",48.27417,16.33581
14486916,99997.gpx,61,"('48.27445', '16.33493')",48.27445,16.33493
14486917,99997.gpx,62,"('48.27461', '16.33503')",48.27461,16.33503
14486918,99997.gpx,63,"('48.27464', '16.33575')",48.27464,16.33575


In [4]:
#cutting away the file extension from 'routes_id' str (it's going to become an index) 
df['route_id'] = df['route_id'].str.split('.').str[0]

#stripping str values from 'num_of_waypoint' (it's going to become column names)
#df['num_of_waypoint'] = df['num_of_waypoint'].str.strip() #seems to be unnecessary
df

,route_id,num_of_waypoint,lat_lgt,latitude,longitude
0,1000303,0,"('38.29842', '-2.65105')",38.29842,-2.65105
1,1000303,1,"('38.29647', '-2.65079')",38.29647,-2.65079
2,1000303,2,"('38.29418', '-2.65011')",38.29418,-2.65011
3,1000303,3,"('38.29499', '-2.65174')",38.29499,-2.65174
4,1000303,4,"('38.29674', '-2.65174')",38.29674,-2.65174
...,...,...,...,...,...
14486915,99997,60,"('48.27417', '16.33581')",48.27417,16.33581
14486916,99997,61,"('48.27445', '16.33493')",48.27445,16.33493
14486917,99997,62,"('48.27461', '16.33503')",48.27461,16.33503
14486918,99997,63,"('48.27464', '16.33575')",48.27464,16.33575


In [5]:
#finding out about the max number of waypoints
df.sort_values(by = 'num_of_waypoint', ascending = False) #--> 3669799.gpx has 118544 waypoints (that hypercorrect documentation of a file should be excluded)

,route_id,num_of_waypoint,lat_lgt,latitude,longitude
13823180,3669799,118544,"('28.91742', '32.34855')",28.91742,32.34855
13823179,3669799,118543,"('28.96985', '32.35542')",28.96985,32.35542
13823178,3669799,118542,"('29.0281', '32.35473')",29.02810,32.35473
13823177,3669799,118541,"('29.05151', '32.35748')",29.05151,32.35748
13823176,3669799,118540,"('29.04461', '32.34752')",29.04461,32.34752
...,...,...,...,...,...
7471459,3463627,0,"('50.45409', '6.37825')",50.45409,6.37825
7471520,3463633,0,"('47.34538', '13.39151')",47.34538,13.39151
7471533,3463653,0,"('61.11827', '14.61835')",61.11827,14.61835
7471653,3463664,0,"('50.99278', '12.43801')",50.99278,12.43801


In [7]:
#exploring the distribution of the total number of waypoints per route a bit more
routes_grp = df.groupby(['route_id'])
routes_grp['num_of_waypoint'].max().median() #--> median of total waypoints per route is 305
routes_grp['num_of_waypoint'].max().describe() #--> 75% is at 509 total waypoints: cutting at 1000 columns will cover most of the routes

count     28032.000000
mean        515.799372
std        1570.527132
min           1.000000
25%         156.000000
50%         305.000000
75%         509.000000
max      118544.000000
Name: num_of_waypoint, dtype: float64

In [8]:
#cutting down the size of the df before pivoting (costly operation)
#would other wise later return "ValueError: Unstacked DataFrame is too big, causing int32 overflow" (been there)

#filtering out any route with more than 1000 waypoints
df = routes_grp.filter(lambda x: x['num_of_waypoint'].max() < 1000) 

In [9]:
df

,route_id,num_of_waypoint,lat_lgt,latitude,longitude
0,1000303,0,"('38.29842', '-2.65105')",38.29842,-2.65105
1,1000303,1,"('38.29647', '-2.65079')",38.29647,-2.65079
2,1000303,2,"('38.29418', '-2.65011')",38.29418,-2.65011
3,1000303,3,"('38.29499', '-2.65174')",38.29499,-2.65174
4,1000303,4,"('38.29674', '-2.65174')",38.29674,-2.65174
...,...,...,...,...,...
14486915,99997,60,"('48.27417', '16.33581')",48.27417,16.33581
14486916,99997,61,"('48.27445', '16.33493')",48.27445,16.33493
14486917,99997,62,"('48.27461', '16.33503')",48.27461,16.33503
14486918,99997,63,"('48.27464', '16.33575')",48.27464,16.33575


In [10]:
#filtering out all routes, where one of the latitudes is too far from Berlin or are wrong data
routes_grp = df.groupby(['route_id']) #new grouping on new df
df = routes_grp.filter(lambda x: x['latitude'].max() > 52.3)
routes_grp = df.groupby(['route_id']) #new grouping on new df
df = routes_grp.filter(lambda x: x['latitude'].min() < 52.7)
df

,route_id,num_of_waypoint,lat_lgt,latitude,longitude
10726,1005019,0,"('52.50607', '13.33208')",52.50607,13.33208
10727,1005019,1,"('52.50553', '13.33163')",52.50553,13.33163
10728,1005019,2,"('52.50525', '13.33148')",52.50525,13.33148
10729,1005019,3,"('52.50515', '13.33337')",52.50515,13.33337
10730,1005019,4,"('52.5052', '13.33366')",52.50520,13.33366
...,...,...,...,...,...
14428518,933359,151,"('52.50444', '13.38246')",52.50444,13.38246
14428519,933359,152,"('52.50525', '13.38633')",52.50525,13.38633
14428520,933359,153,"('52.50643', '13.38615')",52.50643,13.38615
14428521,933359,154,"('52.50648', '13.39023')",52.50648,13.39023


In [11]:
#filtering out all routes, where one of the longitudes is too far from Berlin or are wrong data
routes_grp = df.groupby(['route_id']) #new grouping on new df
df = routes_grp.filter(lambda x: x['longitude'].max() > 12.9)
routes_grp = df.groupby(['route_id']) #new grouping on new df
df = routes_grp.filter(lambda x: x['longitude'].min() < 13.7)
df

,route_id,num_of_waypoint,lat_lgt,latitude,longitude
10726,1005019,0,"('52.50607', '13.33208')",52.50607,13.33208
10727,1005019,1,"('52.50553', '13.33163')",52.50553,13.33163
10728,1005019,2,"('52.50525', '13.33148')",52.50525,13.33148
10729,1005019,3,"('52.50515', '13.33337')",52.50515,13.33337
10730,1005019,4,"('52.5052', '13.33366')",52.50520,13.33366
...,...,...,...,...,...
14428518,933359,151,"('52.50444', '13.38246')",52.50444,13.38246
14428519,933359,152,"('52.50525', '13.38633')",52.50525,13.38633
14428520,933359,153,"('52.50643', '13.38615')",52.50643,13.38615
14428521,933359,154,"('52.50648', '13.39023')",52.50648,13.39023


In [12]:
#reshaping df from long format to wide format
df_reshaped = df.pivot(index='route_id', columns='num_of_waypoint', values='lat_lgt')
df_reshaped

num_of_waypoint,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
route_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1005019,"('52.50607', '13.33208')","('52.50553', '13.33163')","('52.50525', '13.33148')","('52.50515', '13.33337')","('52.5052', '13.33366')","('52.50538', '13.33391')","('52.5053', '13.3342')","('52.50505', '13.33667')","('52.50499', '13.33755')","('52.50504', '13.33822')","('52.50528', '13.3395')","('52.50527', '13.34009')","('52.50518', '13.3407')","('52.50427', '13.34433')","('52.50233', '13.34427')","('52.4999', '13.34294')","('52.49924', '13.34234')","('52.49731', '13.34191')","('52.49525', '13.34169')","('52.48884', '13.34238')","('52.48767', '13.34255')","('52.4873', '13.34182')","('52.48558', '13.34448')","('52.48424', '13.34444')","('52.48349', '13.34423')","('52.48291', '13.34247')","('52.48098', '13.3374')","('52.481', '13.33333')","('52.47988', '13.33277')","('52.47609', '13.33255')","('52.4718', '13.33277')","('52.47149', '13.34204')","('52.47013', '13.34028')","('52.46927', '13.34208')","('52.4694', '13.34397')","('52.465', '13.34431')","('52.46514', '13.34968')","('52.46531', '13.35159')","('52.46178', '13.35322')","('52.46119', '13.35491')","('52.46297', '13.35502')","('52.46297', '13.35442')","('52.46371', '13.35448')","('52.46371', '13.35577')","('52.46426', '13.35577')","('52.46419', '13.35819')","('52.47172', '13.36231')","('52.47371', '13.36352')","('52.47293', '13.36521')","('52.47334', '13.36555')",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113043,"('52.45147', '13.69072')","('52.45147', '13.69075')","('52.45146', '13.69076')","('52.45149', '13.69088')","('52.4515', '13.69103')","('52.45149', '13.6912')","('52.45147', '13.69135')","('52.45146', '13.69141')","('52.45146', '13.69143')","('52.45145', '13.69147')","('52.45136', '13.69152')","('52.45122', '13.69146')","('52.45115', '13.69154')","('52.45093', '13.69141')","('52.45066', '13.69141')","('52.45065', '13.69141')","('52.45064', '13.69141')","('52.45063', '13.69141')","('52.45047', '13.6914')","('52.45043', '13.6914')","('52.45032', '13.69141')","('52.45009', '13.69143')","('52.45008', '13.69143')","('52.45003', '13.69144')","('52.44993', '13.69145')","('52.44992', '13.69145')","('52.44988', '13.69145')","('52.44972', '13.69146')","('52.44974', '13.69145')","('52.44959', '13.6914')","('52.44921', '13.69147')","('52.4491', '13.69148')","('52.44887', '13.69148')","('52.44862', '13.69154')","('52.44844', '13.69151')","('52.44821', '13.69155')","('52.44799', '13.69156')","('52.44769', '13.69153')","('52.44748', '13.69155')","('52.44744', '13.69154')","('52.44723', '13.69155')","('52.447', '13.69158')","('52.44677', '13.69159')","('52.44658', '13.69162')","('52.4464', '13.69153')","('52.44626', '13.69161')","('52.44611', '13.6915')","('52.44612', '13.69143')","('52.44617', '13.69113')","('52.44622', '13.69095')",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113104,"('52.50054', '13.44143')","('52.5009', '13.44212')","('52.50085', '13.44373')","('52.50081', '13.44434')","('52.50087', '13.44462')","('52.50104', '13.44476')","('52.50258', '13.44669')","('52.50499', '13.44003')","('52.50703', '13.43671')","('52.50849', '13.43465')","('52.50991', '13.43066')","('52.51144', '13.4274')","('52.51327', '13.42426')","('52.51431', '13.42139')","('52.51536', '13.41808')","('52.51546', '13.418

In [13]:
#exploring the new shape
df_reshaped.shape # --> (463, 1000), so it's 463 routes with a max of 1000 waypoints

(463, 1000)

Conclusion after Data Cleaning

We had about 29.000 routes parsed from wandermap.net that had showed up with "Berlin" as a search string. After cleaning out all the routes that were too far from Berlin geographically and all the routes with exceptionally detailed waypoints description, we find that only 463 routes are useful enough to be included in our models.


In [14]:
#writing the clean df into a csv file
df_reshaped.to_csv('cleaned_all_routes_data.csv')